In [ ]:
# # from  model_accuracy import *
# from packages import *
# sr_ci_data=create_sr_data()
# incident_ci_data,incident_data=create_incident_data()
# import re

In [1]:
import numpy as np
import pandas as pd
import string
import logging
#import pyLDAvis.gensim
import json
import warnings
from bs4 import BeautifulSoup
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity
from numpy import array
import re
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [2]:
import gensim
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import nltk
#nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [4]:
import collections
import math
import re
from collections import Counter
class OrderedSet(collections.Set):
    def __init__(self, iterable=()):
        self.d = collections.OrderedDict.fromkeys(iterable)

    def __len__(self):
        return len(self.d)

    def __contains__(self, element):
        return element in self.d

    def __iter__(self):
        return iter(self.d)
    
WORD = re.compile(r"\w+")

def text_to_vector(text):
    words = WORD.findall(text.upper())
    return Counter(words)

def get_cosine(vec1, vec2):
    vec1 = text_to_vector(vec1)
    vec2 = text_to_vector(vec2)
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

    

In [5]:
def dbcar_split(df):
    
    #suspected CI name based correction 
    
    def fix_CI_Name(sus_CI_Name):
        
        if str(sus_CI_Name).strip() in ['dbCAR-cRDS','dbCAR-cRDS_PRD']:
            return 'dbCAR-cRDS-dbCAR-cRDS'
        
        elif sus_CI_Name.strip() in ['db-CLM','db-CLM-(Global)','db-CLM_PRD']:
            return 'dbCAR-cRDS-db-CLM'
        
        else:
            return 'dbCAR-cRDS-other'
        
    df['Incident CI Name'] = df['Incident Suspected Cause CI Name'].apply(fix_CI_Name)
    
    # SD based correction
    
    df_1 = df[df['Incident Short Description'].str.contains(pat='[d|D][b|B]-*[c|C][l|L][m|M]',regex=True)]
    df_2 =df[~(df['Incident Short Description'].str.contains(pat='[d|D][b|B]-*[c|C][l|L][m|M]',regex=True))]
    
    df_1['Incident CI Name'] = 'dbCAR-cRDS-db-CLM'
    
    final_df = pd.concat([df_1,df_2])
    
    return final_df

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [7]:
# def get_html_to_text_format(html_string):
#     soup = BeautifulSoup(html_string)
#     out = soup.get_text()
#     out =" ".join(out.split())
#     out = " ".join(out.split('x000D'))
#     return out
# def remove_punctuation(text):
#     new_text=''.join([char for char in text if char not in string.punctuation])
#     return new_text
# rem = string.punctuation
# pattern = r"[{}]".format(rem)

 

# def datapreprocessing(df,data):
#     #remove punctvation
#     df[data]=df[data].str.replace(pattern, ' ')
#     #punctuation removing
#     df[data]  = df[data] .map(lambda x: x.lower())
#     #Remove digits
#     df[data] = df[data].str.replace('\d+', '')
#     #Remove one and two letter words
#     #df[data] = df[data].str.replace(r'\b(\w{1,2})\b', '')
#     return df[data]
# def remove_stopwords(texts):
#     return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]  
# def Cleaning(df,col):
#     df[col] = datapreprocessing(df,col)
#     df[col] = df[col].str.lower().str.split()
#     df[col] = df[col].apply(lambda x: [item for item in x if item not in stop_words])
#     df[col] = df[col].apply(lambda x: ', '.join(map(str, x)))
#     df[col] = df[col].apply(lambda x: ''.join(x.split(',')))
#     return df[col]

In [8]:
dic = {"pls": "please"}
#Update as per comment
dic = {r"\b{}\b".format(k): v for k, v in dic.items()}
def get_html_to_text_format(html_string):
    soup = BeautifulSoup(html_string)
    out = soup.get_text()
    out =" ".join(out.split())
    out = " ".join(out.split('x000D'))
    return out


stop = stopwords.words('english')
stop_words = stop  
stop_words = stop_words+ ['january','february','march','april','may','june','july','august','september','october','november','december',
'jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec','sunday','monday','tuesday','wednesday','thursday',
'friday','saturday','sun','mon','tue','wed','thu','fri','sat','id','ids','isin','top','re','from']

stop_words.remove("not")


def preprocess(raw_text):
    
    """Convert all characters to lowercase from raw text"""
    words_ = str(raw_text).lower().split()

    

    """Remove stop words from list of tokenized words"""
    cleaned_words = []
    for word in words_:
        if word not in stop_words:
            cleaned_words.append(word)
        
            
    months = ['january','jan',
          'february','feb',
          'march','mar', 
          'april','apr',
          'may',
          'june','jun',
          'july','jul',
          'august','aug',
          'september','sep',
          'october','oct',
          'november','nov',
          'december','dec']        
            
    matching = [s for s in cleaned_words if any(xs in s for xs in months)]   
    if len(matching)>=1:
        for i in range(len(matching)):
            cleaned_words.remove(matching[i])
    

            
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in cleaned_words:
        new_word = re.sub(r'[^a-zA-Z0-9]',' ', word)
        if new_word != ' ':
            new_words.append(new_word)   
            
            
            
    """Remove only digits from list of tokenized words"""
    new_digits = []
    for word in new_words:
        new_digit = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$",'',word)
        if new_digit != '':
            new_digits.append(new_digit)             
                   
    """Remove hexa decimal from list of tokenized words"""
    
    non_hexa_texts = []
    for word in new_digits:
        non_hexa_text = re.sub(r'[^\x00-\x7f]','', word)
        if non_hexa_text != '':
            non_hexa_texts.append(non_hexa_text) 
                
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in cleaned_words:
        new_word = re.sub(r'[^a-zA-Z0-9]',' ', word)
        if new_word != ' ':
            new_words.append(new_word)   
            
            
            
    """Remove only digits from list of tokenized words"""
    new_digits = []
    for word in new_words:
        new_digit = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$",'',word)
        if new_digit != '':
            new_digits.append(new_digit)             
                   
    """Remove hexa decimal from list of tokenized words"""
    
    non_hexa_texts = []
    for word in new_digits:
        non_hexa_text = re.sub(r'[^\x00-\x7f]','', word)
        if non_hexa_text != '':
            non_hexa_texts.append(non_hexa_text) 
            
    output =  " ".join(non_hexa_texts) 
    dic = {"pls": "please"}
    for word, initial in dic.items():
        output = output.replace(word.lower(), initial)
    output = re.sub(' +', ' ',output)
    output = output.strip()   
    return output
            


In [9]:
import sys
sys.path.append("/home/jupyter/IB_Tech_AI_Ops/Ravi")

In [ ]:
# #only 2020
# from packages import *
# incident_ci_data,incident_data=create_incident_data()
# incident_ci_data = dbcar_split(incident_ci_data)

In [10]:
#2019 include
df = pd.read_csv('gs://db_input_data/Cleansed Files - Dev/2019_2020_merged.csv')
incident_ci_data= dbcar_split(df)

In [11]:
df = incident_ci_data[incident_ci_data['Incident CI Name']=='dbCAR-cRDS-dbCAR-cRDS']

In [12]:
df.head(1)

,Unnamed: 0,Incident Reference Number,Incident State,Incident CI ID,Incident Priority,Incident Urgency,Incident Impact,Incident Risk ?,Incident Start Date & Time,Incident End Date & Time,Incident Symptom Code,Incident Closure Code,Closure Code 2,Solution Code,Incident Suspected Cause CI ID,Incident Assignment Group,Incident Knowledge Article Reference,Incident Linked Problem Reference Number,Incident Recovered Date & Time,Incident Reassignment Count,Incident Short Description,Incident Resolution Description,Incident Causing Change,Incident Solving Change,Incident Contact Type,Incident Resolved Date & Time,Problem Number,Problem Root Cause Code,Problem Root Cause Code 2,Problem Root Cause Code 3,Incident CI Name,Incident Suspected Cause CI Name,Incident Priority-Urgency,High Priority-Urgency Incident Flag,Incident Start Date,Incident Start Year,Incident Start Month,Incident Start WeekNumber,Incident Start Hour,MTTR_days,MTTR_categories,MTTR_Hours,MTTR_Hours_Categories,Days Since Last Incident,Incident Active Last 30 Days,Is Incident Caused by Change
17,17,INC0014625038,Closed,59248-1,Priority 3,High,Low,No,2020-01-01 01:41:00,2020-01-06 05:40:57,Monitoring,Configuration,App-Operating System Config,Amend Configuration,59248-1,CLM_ENGINEERING,KB0287159,NaN,2020-01-06 05:53:59,0.0,Delay in Publishing,Under analysis with CLM Engg and MIS Dev. _x000D_Please refer to https://dbatlas.db.com/jira02/browse/CLMPS-322 for further details.,NaN,NaN,Self-service,2020-01-06 05:53:59,NaN,NaN,NaN,NaN,dbCAR-cRDS-dbCAR-cRDS,dbCAR-cRDS,Priority 3-High,Low Priority-Urgency,2020-01-01,2020,1,1,1,5.0,3-17 days,124.0,>96 hours,485 days 16:42:49,Not Active Last 30 days,Yes


In [13]:
df.columns

Index(['Unnamed: 0', 'Incident Reference Number', 'Incident State',
       'Incident CI ID', 'Incident Priority', 'Incident Urgency',
       'Incident Impact', 'Incident Risk ?', 'Incident Start Date & Time',
       'Incident End Date & Time', 'Incident Symptom Code',
       'Incident Closure Code', 'Closure Code 2', 'Solution Code',
       'Incident Suspected Cause CI ID', 'Incident Assignment Group',
       'Incident Knowledge Article Reference',
       'Incident Linked Problem Reference Number',
       'Incident Recovered Date & Time', 'Incident Reassignment Count',
       'Incident Short Description', 'Incident Resolution Description',
       'Incident Causing Change', 'Incident Solving Change',
       'Incident Contact Type', 'Incident Resolved Date & Time',
       'Problem Number', 'Problem Root Cause Code',
       'Problem Root Cause Code 2', 'Problem Root Cause Code 3',
       'Incident CI Name', 'Incident Suspected Cause CI Name',
       'Incident Priority-Urgency', 'High Prio

In [14]:
df = df[['Incident Reference Number','Incident Knowledge Article Reference',
         'Incident Short Description','Incident Symptom Code','Incident Start Year']]

In [15]:
df['Incident Symptom Code']=df['Incident Symptom Code'].apply(lambda x: "Others" if x in ['Connectivity Issue','Performance Issue','Not Listed','Batch Issue','File transfer','Backup Issue','Security/Virus','Performance','General Error','File Transfer'] else x)

In [16]:
df['Incident Short Description'] = df['Incident Short Description'].apply(lambda x:preprocess(x))


In [17]:
# df['Incident Short Description']= Cleaning(df,'Incident Short Description')

In [18]:
df = df[df['Incident Symptom Code'] == 'Application Function Issue'].reset_index(drop=True)

In [19]:
df['Incident Short Description'].nunique()

9308

In [20]:
df.shape

(14584, 5)

In [21]:
df = df[(df['Incident Short Description']!='missing') & (df['Incident Short Description']!=" ")].reset_index(drop=True)

In [22]:
df = df.reset_index(drop=True)

In [23]:
# df_2019=pd.DataFrame(df[df['Incident Start Year']==2019]['Incident Short Description'].drop_duplicates()).reset_index(drop=True)
# df_2019.to_csv('incident_2019.csv')

In [24]:
sentences=list(df['Incident Short Description'])

In [25]:
tokenized_sent = []
for s in sentences:
    tokenized_sent.append(word_tokenize(s))
tokenized_sent[0]

['broken', 'pipe', 'error', 'crds', 'ms2', 'lonindapnp524', 'uk', 'db', 'com']

In [26]:
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_sent)]
tagged_data[0:2]

[TaggedDocument(words=['broken', 'pipe', 'error', 'crds', 'ms2', 'lonindapnp524', 'uk', 'db', 'com'], tags=[0]),
 TaggedDocument(words=['mifid', 'classification', 'can', 't', 'modified', 'cer', 'appearing', 'modification'], tags=[1])]

In [27]:
#model = Doc2Vec(tagged_data, vector_size = 100, window = 3, min_count = 2, epochs = 100)
model = Doc2Vec(tagged_data, vector_size = 300, window = 3,epochs = 500,dm=0)
'''
vector_size = Dimensionality of the feature vectors.
window = The maximum distance between the current and predicted word within a sentence.
min_count = Ignores all words with total frequency lower than this.
alpha = The initial learning rate.
'''

'\nvector_size = Dimensionality of the feature vectors.\nwindow = The maximum distance between the current and predicted word within a sentence.\nmin_count = Ignores all words with total frequency lower than this.\nalpha = The initial learning rate.\n'

In [28]:
# Save model
filename = 'doc_model.wv' # Can be any arbitrary filename
model.save(filename) 
# Reload model
retrieved_model = gensim.models.doc2vec.Doc2Vec.load(filename)


In [29]:
i= "internal error removal pr action nca npe  dbCLM pipeline errors"
j="Hi I am ravi"

In [31]:
n=10
test_doc = word_tokenize(i.lower())
test_doc_vector = model.infer_vector(test_doc)
model.docvecs.most_similar(positive = [test_doc_vector],topn=n)

[(6331, 0.8324095010757446),
 (6334, 0.6411031484603882),
 (5362, 0.592831552028656),
 (4827, 0.5809881687164307),
 (8541, 0.5674904584884644),
 (3842, 0.5569115877151489),
 (2997, 0.5475839376449585),
 (4693, 0.5367885828018188),
 (5478, 0.5278290510177612),
 (5063, 0.5234280824661255)]

In [35]:
test= pd.read_csv("/home/jupyter/IB_Tech_AI_Ops/Ravi/incident_queries_2019_2020_test_data.csv")

In [38]:
test.columns=['query','mapping','sample_type']

In [39]:
# test['query']= Cleaning(test,'query')

In [40]:
n=20
df_final=[]
for i in test['query']:
    test_doc = word_tokenize(i.lower())
    test_doc_vector = model.infer_vector(test_doc)
    model.docvecs.most_similar(positive = [test_doc_vector],topn=n)
    df_1= pd.DataFrame(model.docvecs.most_similar(positive = [test_doc_vector],topn=n),columns=['index','sim'])
    df_1['query']=i
    df_2= df.join(df_1.set_index('index'),how='inner').reset_index(drop=True)
    df_final.append(df_2)
df_final=pd.concat(df_final)
#df_final.drop('index',inplace=True)

In [41]:
df_final.shape

(5500, 7)

In [43]:
df_final.drop_duplicates(['Incident Short Description','query'],inplace=True)

In [44]:
df_final.shape

(3305, 7)

In [45]:
# df_final['query'].value_counts()

In [46]:
df_final['rank'] = df_final.groupby(['query'])[['sim']].rank(method='first',ascending=False)
df_final=df_final.sort_values(['query','rank'],ascending=[True,True])
df_final.reset_index(drop=True,inplace=True)

In [47]:
df_final=df_final.merge(test,on='query',how='inner')

In [48]:
df_final['Is Correct Inc Recommendation']=df_final.apply(lambda row: 1 if (row['Incident Short Description']==row['mapping'] and row['sim']>0 ) else 0, axis=1)

In [49]:
df.shape

(14546, 5)

In [50]:
df_final.shape

(3305, 11)

In [ ]:
#0

In [ ]:
top1 = df_final[df_final['rank']<2]
top2 = df_final[df_final['rank']<3]
top3 = df_final[df_final['rank']<4]

In [ ]:
top1.shape,top2.shape,top3.shape

In [ ]:
top1_per=top1['Is Correct Inc Recommendation'].sum() / top1.shape[0]

In [ ]:
top1_per

In [ ]:
#top1[top1['Is Correct Inc Recommendation']==0][['mapping','Incident Short Description','sample_type']]
# # #rightIndex-[6,9,15,24,33,162,174,210,234,267,279,291,399,486,504,507]

In [ ]:
# top1_manuval = [6,9,15,24,33,162,174,210,234,267,279,291,399,486,504,507]

In [ ]:
top1_rightquery=top1[top1['Is Correct Inc Recommendation']==1]['query'].tolist()

In [ ]:
# top1_rightquery_manuval= top1[top1.index.isin(top1_manuval)]['query'].tolist()
# top1_rightquery=top1_rightquery+top1_rightquery_manuval

In [ ]:
len(top1_rightquery)

In [ ]:
top2 = top2[~top2['query'].isin(top1_rightquery)]

In [ ]:
top2_per=top2['Is Correct Inc Recommendation'].sum() / (top1.shape[0])

In [ ]:
top2_per

In [ ]:
top2_rightquery=top2[top2['Is Correct Inc Recommendation']==1]['query'].unique().tolist()

In [ ]:
len(top2_rightquery)

In [ ]:
#top1=
#top2=3
#top2 manu 

In [ ]:
# top2_manuval=[]

In [ ]:
# top2[(top2['Is Correct Inc Recommendation']==0)&(top2['rank']==2)][['mapping','Incident Short Description','sample_type']]

In [ ]:
# top2_rightquery_manuval= top2[top2.index.isin(top2_manuval)]['query'].tolist()
# top2_rightquery = top2_rightquery+top2_rightquery_manuval

In [ ]:
#adding top2 mappings manuval and automate


In [ ]:
#Adding top1 and top2 mappings
top2_rightquery=top2_rightquery+top1_rightquery

In [ ]:
len(top2_rightquery)

In [ ]:
top3 = top3[~top3['query'].isin(top2_rightquery)]

In [ ]:
top3_per=top3['Is Correct Inc Recommendation'].sum() / (top1.shape[0])

In [ ]:
top3_per

In [ ]:
top3_rightquery=top3[top3['Is Correct Inc Recommendation']==1]['query'].unique().tolist()

In [ ]:
len(top3_rightquery)

In [ ]:
top3_rightquery=top3_rightquery+top2_rightquery

In [ ]:
len(top3_rightquery)

In [ ]:
# top3_manuval=[]
# #

In [ ]:
# top3[(top3['Is Correct Inc Recommendation']==0)&(top3['rank']==3)][['mapping','Incident Short Description','sample_type']]

In [ ]:
round(top1_per,2),round(top1_per+top2_per,2),round(top1_per+top2_per+top3_per,2)

In [ ]:
len(top1_rightquery),len(top2_rightquery),len(top3_rightquery)

In [ ]:
#After Dropping duplicates accuracyincreases 55,64,67

In [ ]:
#Manuval
#111,118,121
#0.63,0.67,0.69
